In [1]:
import jieba
import re
from sklearn import model_selection
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,classification_report

In [2]:
def read_stop(path):
    stop_word=[line.strip() for line in open(path,'r',encoding='UTF-8').readlines()]
    return stop_word

In [3]:
stop_word=read_stop("stop.txt")

In [4]:
def load_data(path):
    lbs=[]
    sentences=[]
    with open(path,'r',encoding='utf8') as f:
        for line in f.readlines():
            x=line.split('_!_')
            lbs.append(x[1])
            sentence = re.sub("[\s+\.\!\/_,$%^*(+\"\')]+|[+——()?【】“”！，。？、~@#￥%……&*（）《》：]+", "",x[3]) #去除标点符号
            words=jieba.lcut(sentence)
            sentences.append(words)
        f.close()
    return sentences,lbs

In [5]:
sentences,lbs=load_data("news_classify_data.txt")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lazybug\AppData\Local\Temp\jieba.cache
Loading model cost 0.557 seconds.
Prefix dict has been built successfully.


In [6]:
def get_dict(sentences,stop_word):
    word_dict={}
    for sentence in sentences:
        for word in sentence:
            if word!='' and word.isalpha(): 
                if word not in stop_word:
                    word_dict[word]=word_dict.get(word,1)+1
    
    word_dict=sorted(word_dict.items(),key=lambda x:x[1],reverse=True)
    return word_dict[:10000]

In [7]:
word_dict=get_dict(sentences,stop_word)

In [10]:
def get_first(arr_list):
    first_list=[]
    for i in arr_list:
        first_list.append(i[0])
    return first_list

In [12]:
word_dict=get_first(word_dict)

In [14]:
def to_onehot(sentence,word_dict):
    return [1 if word in sentence else 0 for word in word_dict]

In [15]:
def get_onehot(sentences,word_dict):
    return [to_onehot(sentence,word_dict) for sentence in sentences]

In [17]:
sentences=get_onehot(sentences,word_dict)

In [20]:
train_x,test_x,train_y,test_y=model_selection.train_test_split(sentences,lbs,test_size=0.1)

In [30]:
model=MultinomialNB()

In [31]:
model.fit(train_x,train_y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [34]:
print(model.score(test_x,test_y))
pred=model.predict(test_x)
print(classification_report(test_y,pred))

0.7737110680978356
             precision    recall  f1-score   support

          0       0.78      0.71      0.74       505
          1       0.76      0.85      0.80       577
          2       0.88      0.82      0.85       455
          3       0.64      0.63      0.63       780
          4       0.81      0.80      0.80       370
          5       0.87      0.85      0.86       783
          6       0.82      0.86      0.84       820
          7       0.69      0.69      0.69       585
          8       0.78      0.70      0.74       434
          9       0.73      0.81      0.77       374

avg / total       0.77      0.77      0.77      5683

